In [1]:
pip install h2o

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, classification_report
from sklearn.model_selection import train_test_split

import h2o
from h2o.frame import H2OFrame
from h2o.estimators.random_forest import H2ORandomForestEstimator

%matplotlib inline

In [3]:
data = pd.read_csv('./Fraud_Data.csv', parse_dates=['signup_time', 'purchase_time'])
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [4]:
address2country = pd.read_csv('./IpAddress_to_Country.csv')
address2country.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [5]:
# Merge the two datasets and print the first 5 rows

In [6]:
# Fonction pour trouver le pays en fonction de l'adresse IP
def find_country(ip):
    row = address2country[(address2country['lower_bound_ip_address'] <= ip) & (address2country['upper_bound_ip_address'] >= ip)]
    if not row.empty:
        return row.iloc[0]['country']
    return 'NA'

# Appliquer la fonction à la colonne 'ip_address'
data['country'] =data['ip_address'].apply(find_country)

# Afficher les 5 premières lignes du jeu de données fusionné
print(data.head())

   user_id         signup_time       purchase_time  purchase_value  \
0    22058 2015-02-24 22:55:49 2015-04-18 02:47:11              34   
1   333320 2015-06-07 20:39:50 2015-06-08 01:38:54              16   
2     1359 2015-01-01 18:52:44 2015-01-01 18:52:45              15   
3   150084 2015-04-28 21:13:25 2015-05-04 13:54:50              44   
4   221365 2015-07-21 07:09:52 2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class        country  
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0          Japan  
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0  United States  
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1  United States  
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0             NA  
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0  United States  


### Ingénierie des fonctionnalités

In [7]:
# Calculer le décalage horaire en secondes
data['time_diff'] = (data['purchase_time'] - data['signup_time']).dt.total_seconds()

# Afficher les 5 premières lignes pour vérifier
print(data[['signup_time', 'purchase_time', 'time_diff']].head())


          signup_time       purchase_time  time_diff
0 2015-02-24 22:55:49 2015-04-18 02:47:11  4506682.0
1 2015-06-07 20:39:50 2015-06-08 01:38:54    17944.0
2 2015-01-01 18:52:44 2015-01-01 18:52:45        1.0
3 2015-04-28 21:13:25 2015-05-04 13:54:50   492085.0
4 2015-07-21 07:09:52 2015-09-09 18:40:53  4361461.0


##### Vérifier si l'Identifiant de l'Appareil est Unique

In [8]:
# Compter le nombre d'utilisateurs par identifiant de l'appareil
device_counts = data['device_id'].value_counts()

# Créer une nouvelle colonne indiquant si l'identifiant de l'appareil est unique
data['device_num'] = data['device_id'].map(device_counts)

# Afficher les 5 premières lignes pour vérifier
print(data[['device_id', 'device_num']].head())

       device_id  device_num
0  QVPSPJUOCKZAR           1
1  EOGFQPIZPYXFZ           1
2  YSSKYOSJHPPLJ          12
3  ATGTXKYKUDUQN           1
4  NAUITBZFJKHWW           1


#### Vérifier si l'Adresse IP est Partagée par Plusieurs Utilisateurs

In [9]:
# Compter le nombre d'utilisateurs par adresse IP
ip_counts = data['ip_address'].value_counts()

# Créer une nouvelle colonne indiquant si l'adresse IP est partagée
data['ip_num'] = data['ip_address'].map(ip_counts)

# Afficher les 5 premières lignes pour vérifier
print(data[['ip_address', 'ip_num']].head())


     ip_address  ip_num
0  7.327584e+08       1
1  3.503114e+08       1
2  2.621474e+09      12
3  3.840542e+09       1
4  4.155831e+08       1


#### Extraire la Semaine de l'Année et le Jour de la Semaine

In [10]:
# Extraire le jour de la semaine et la semaine de l'année pour l'heure d'inscription
data['signup_day'] = data['signup_time'].dt.dayofweek
data['signup_week'] = data['signup_time'].dt.isocalendar().week

# Extraire le jour de la semaine et la semaine de l'année pour l'heure d'achat
data['purchase_day'] = data['purchase_time'].dt.dayofweek
data['purchase_week'] = data['purchase_time'].dt.isocalendar().week

# Calculer le décalage horaire en secondes 
data['time_diff'] = (data['purchase_time'] - data['signup_time']).dt.total_seconds()

# Afficher les 5 premières lignes pour vérifier
print(data[['time_diff', 'signup_day', 'signup_week', 'purchase_day', 'purchase_week']].head())

   time_diff  signup_day  signup_week  purchase_day  purchase_week
0  4506682.0           1            9             5             16
1    17944.0           6           23             0             24
2        1.0           3            1             3              1
3   492085.0           1           18             0             19
4  4361461.0           1           30             2             37


In [11]:
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,time_diff,device_num,ip_num,signup_day,signup_week,purchase_day,purchase_week
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,1,1,1,9,5,16
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,1,1,6,23,0,24
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,1.0,12,12,3,1,3,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,492085.0,1,1,1,18,0,19
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,1,1,1,30,2,37


In [12]:
# Define features and target to be used

#### Définir les Fonctionnalités et la Cible :

In [13]:
# Définir les fonctionnalités (features) et la cible (target)
features = ['signup_day', 'signup_week', 'purchase_day', 'purchase_week', 'purchase_value', 'source', 'browser', 'sex', 'age', 'country', 'time_diff', 'device_num', 'ip_num']
target = 'class'

# Sélectionner les fonctionnalités et la cible dans le jeu de données
X = data[features]
y = data[target]

# Afficher les 5 premières lignes pour vérifier
print(X.head())
print(y.head())

   signup_day  signup_week  purchase_day  purchase_week  purchase_value  \
0           1            9             5             16              34   
1           6           23             0             24              16   
2           3            1             3              1              15   
3           1           18             0             19              44   
4           1           30             2             37              39   

  source browser sex  age        country  time_diff  device_num  ip_num  
0    SEO  Chrome   M   39          Japan  4506682.0           1       1  
1    Ads  Chrome   F   53  United States    17944.0           1       1  
2    SEO   Opera   M   53  United States        1.0          12      12  
3    SEO  Safari   M   41             NA   492085.0           1       1  
4    Ads  Safari   M   45  United States  4361461.0           1       1  
0    0
1    0
2    1
3    0
4    0
Name: class, dtype: int64


In [14]:
# Split into 70% training and 30% test dataset
# Define features and target

##### Diviser les Données et Définir les Fonctionnalités et la Cible

In [15]:
# Diviser les données en ensembles d'entraînement (70%) et de test (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Afficher la forme des ensembles d'entraînement et de test
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(105778, 13) (45334, 13) (105778,) (45334,)


In [16]:
# Build random forest model 

In [17]:
# import des biblitothèque
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score


#### Construire le Modèle de Forêt Aléatoire

In [18]:
# Instancier le classificateur de forêt aléatoire
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Ajuster le modèle aux données d'entraînement
rf_model.fit(X_train, y_train)

ValueError: could not convert string to float: 'SEO'